## MXNet Optimizer Tutorial

Deep learning is used in lots of different practical applications from image classification to machine translation to speech recognition and many more. While, there are different deep neural model architectures for handling these various applications, from Convolutional Neural Networks for image-based tasks, to Recurrent Neural Networks for tasks with sequential inputs, one thing remains constant - training a deep neural model requires solving an optimization problem, and this is usually acheived with a gradient descent algorithm that iteratively changes the parameters (or weights) of the model.

Put simply, all supervised deep learning models parametrize a function from inputs to output. In order to learn the value of the parameters, we start with an initialization scheme and iteratively refine the parameter initial values by moving along a direction that is opposite to the (approximate) gradient of the loss function. This functionality is abstracted by the Optimizer API in MXNet. A single step of iterative refinement of the model parameters is achieved in MXNet by calling `optimizer.step`. This tutorial provides an overview of the various optimizers built-in to MXNet and how to use them.

### Stochastic Gradient Descent
The most commonly used gradient descent algorithm for deep learning tasks is a generalization of stochastic gradient descent. Technically, stochastic gradient descent (sgd) refers to an online approximation of the gradient descent algorithm characterized by calculating the gradient of the loss function applied to a single datapoint, instead of your entire dataset, and using that to update your parameter values. However, in MXNet, and other deep learning frameworks, the sgd optimizer is agnostic to how many datapoints the loss function is applied to and it is quite common to have the sgd update influenced by a mini-batch loss gradient instead of a single datapoint loss gradient. Furthermore, all other MXNet optimizers are based on sgd, with a modified update step.

#### sgd optimizer
The sgd optimizer object accepts a construction parameter referred to as the learning rate. This learning rate term is used to determine the step size of the update in the direction opposite to the calculated gradient of the loss function. 

For sgd with learning rate `lr`, the optimizer.step function performs the single update step:

$$w_{i+1} = w_i + lr\cdot -grad(w_i)$$

visualized in the diagram shown below


<p align="center">
    <img src="sgd_take_six.gif" alt="drawing"/>
</p>


#### sgd optimizer with weight decay
The sgd update step can be augmented with an extra term to introduce a penalty on the size of the parameters to ensure small weights. Introducing weight decay modifies the objective of the optimization problem implicitly with a regularization term that penalizes large weights.

For sgd with learning rate `lr` and weight decay `wd` the optimizer.step function performs the single update step:

$$w_{i+1} = w_i + lr\cdot (-grad(w_i) -\texttt{wd}\cdot w_i)$$

#### sgd optimizer with momentum
The convergence of the sgd optimizer can also be accelerated by momentum. The goal of sgd with momentum is to stabilize the convergence of sgd by improving the approximation of the gradient term by including the gradient terms from previous update steps. In order to achieve this sgd with momentum remembers the update at each iteration to be included in the next iteration and in the equations below we denote the momentum history as $v$. sgd with momentum was introduced by Rumelhart, Hinton and Williams.

For the first update the sgd optimizer with momentum performs the single update step:

$$ v_1= lr\cdot -grad(w_0) \\  
 w_1= w_0 + v_1 $$

For subsequent updates, sgd with momentum with momentum parameter $\gamma$ performs the update step:

$$ v_{i+1} = \gamma \cdot v_{i} + lr\cdot -grad(w_{i}) \\
w_{i+1} = w_i + v_{i+1} $$


<p align="center">
    <img src="momentum_sgd_take_five.gif" alt="drawing"/>
</p>

### Nesterov Accelerated Stochastic Gradient Descent

The momentum method of Nesterov and Nemirovski is a modification to stochastic gradient descent with momentum that allows for faster convergence in practice. With Nesterov Accelerated gradient (nag) descent, the gradient of the loss function is **not** taken with respect to the current parameters as in sgd, but with respect to the refined parameter values after an sgd update step, where the current momentum is used as an estimate for the update term.

In other words, with the nag optimizer there are two update steps. The first update step uses the momentum as an approximation of the negative gradient to derive new values for the weights $(w_i + \gamma \cdot v_i)$. This is known as the lookahead step. The next update step uses the gradient of the loss function with respect to this lookahead parameter value, and the momentum to obtain a new direction and refine our original parameter values, like regular sgd with momentum.

The nag optimizer with momentum parameter $\gamma$ performs the update step:

$$ v_{i+1} = \gamma \cdot v_{i} + lr\cdot -grad(w_{i} + \gamma \cdot v_i)\\  
w_{i+1} = w_i + v_{i+1} $$

<p align="center">
    <img src="nesterov_momentum_take_three.gif" alt="drawing"/>
</p>

### Adaptive Learning Rate Methods

### AdaGrad

The AdaGrad optimizer maintains 

$$ G_{i+1} = G_{i} + [grad(w_i)\cdot grad(w_i)^\top] $$
$$ w_{i+1} = w_i + \dfrac{lr}{\sqrt{\epsilon\cdot I + diag(G_{i+1})}}\cdot -grad(w_i)$$

Duchi et al

Rewriting we have
$$ g^2_{i+1} = g^2_{i} + grad(w_i)^2 $$
$$ w_{i+1} = w_i + \dfrac{lr}{\sqrt{g^2 + \epsilon}}\cdot -grad(w_i)$$


### RMSProp

The RMSProp optimizer keeps a moving average of the derivative of the loss function with respect to the parameters.


$$ \mathbb{E}[g^2]_{i+1} = \beta\cdot\mathbb{E}[g^2]_{i} + (1-\beta)\cdot [grad(w_{i})]^2 \\
 w_{i+1} = w_i + \dfrac{lr}{\sqrt{\mathbb{E}[g^2]_{i+1} + \epsilon}}\cdot -grad(w_i)$$


#### RMSProp (Centered)
$$ \mathbb{E}[g]_{i+1} = \beta\cdot\mathbb{E}[g]_{i} + (1-\beta)\cdot [grad(w_{i})] \\
 \mathbb{E}[g^2]_{i+1} = \beta\cdot\mathbb{E}[g^2]_{i} + (1-\beta)\cdot [grad(w_{i})]^2 \\
 v_{i+1} = \gamma \cdot v_{i} + \dfrac{lr}{\sqrt{\mathbb{E}[g^2]_{i+1} - \mathbb{E}[g]^2_{i+1}+ \epsilon}}\cdot -grad(w_{i}) \\
 w_{i+1} = w_i + v_{i+1} \\ $$


Hinton lecture notes, 
Graves

### AdaDelta
The AdaGrad optimizer maintains 

$$ \mathbb{E}[\Delta w^2]_{i+1} = \beta\cdot\mathbb{E}[\Delta w^2]_i + (1 - \beta) \cdot (w_i - w_{i-1})^2 \\
 \mathbb{E}[g^2]_{i+1} = \beta\cdot\mathbb{E}[g^2]_{i} + (1-\beta)\cdot [grad(w_{i})]^2 \\
 w_{i+1} = w_i + \dfrac{\sqrt{\mathbb{E}[\Delta w^2] + \epsilon}}{\sqrt{\mathbb{E}[g^2]_{i+1} + \epsilon}} \cdot -grad(w_i)$$

### Adam
The Adam optimizer maintains 
$$ v_{i+1} = \beta_1 \cdot v_{i} + (1 - \beta_1) \cdot grad(w_i) \\
 \mathbb{E}[g^2]_{i+1} = \beta_2\cdot\mathbb{E}[g^2]_{i} + (1-\beta_2)\cdot [grad(w_{i})]^2 \\
 \tilde{v}_{i+1} = \dfrac{v_{i+1}}{1 - \beta_1^{i+1}} \\
 \tilde{\mathbb{E}[g^2]}_{i+1} = \dfrac{\mathbb{E}[g^2]_{i+1}}{1 - \beta_2^{i+1}} \\
 w_{i+1} = w_i + \dfrac{lr}{\sqrt{\tilde{\mathbb{E}[g^2]}_{i+1}} + \epsilon} \cdot -\tilde{v}_{i+1}$$
 
 
 Note: Check whether adam is implemented with correction

### AdaMax
The Adam optimizer maintains 
$$ v_{i+1} = \beta_1 \cdot v_{i} + (1 - \beta_1) \cdot grad(w_i) \\
 g^\infty_{i+1} = \mathtt{max}(\beta_2\cdot g^\infty_{i},  |{grad(w_i)}|) \\
 \tilde{v}_{i+1} = \dfrac{v_{i+1}}{1 - \beta_1^{i+1}} \\
 w_{i+1} = w_i + \dfrac{lr}{g_{i+1} + \epsilon} \cdot - \tilde{v}_{i+1}$$

### NAdam
The NAdam optimizer maintains 
$$ v_{i+1} = \beta_1 \cdot v_{i} + (1 - \beta_1) \cdot grad(w_i + \beta_1 \cdot v_{i}) \\
 \mathbb{E}[g^2]_{i+1} = \beta_2\cdot\mathbb{E}[g^2]_{i} + (1-\beta_2)\cdot [grad(w_{i})]^2 \\
 \tilde{v}_{i+1} = \dfrac{v_{i+1}}{1 - \beta_1^{i+1}} \\
 \tilde{\mathbb{E}[g^2]}_{i+1} = \dfrac{\mathbb{E}[g^2]_{i+1}}{1 - \beta_2^{i+1}} \\
 w_{i+1} = w_i + \dfrac{lr}{\sqrt{\tilde{\mathbb{E}[g^2]}_{i+1}} + \epsilon}\cdot - \tilde{v}_{i+1}$$

### Further Approximations on SGD

### Signum

### SGLD

### Online Learning Algorithms

### FTML

### FTRL

### SGD optimized for distributed training

### LBSGD

### DCASGD